# Imports

In [7]:
from __future__ import annotations

import os
from importlib import reload
import sys; sys.path.append('../../_Common')
import amara; reload(amara)

from joblib import Parallel, delayed
from datetime import datetime

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from amara.static.groupings import taxes, breakfast_groups
from amara.core.grouping import group_categories, group_thresholds
from amara.core.wrappers import DirectoryWrapper, DataFrameWrapper

from amara.datasets.Info_HMS_Raw_Arrivals import mend_arrival_departure_dates

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# read data and consolidate
filepaths = DirectoryWrapper(f'../../Raw Data/Info_HMS_Raw_Arrivals').files
dfs: list[pd.DataFrame] = Parallel(n_jobs=-1, verbose=0)(delayed(pd.read_excel)(filepath) for filepath in filepaths)
InfoHMS_consolidated = pd.concat(dfs).drop_duplicates('Confirmation Number')

KeyboardInterrupt: 

In [ ]:
extracted = InfoHMS_consolidated.copy(deep=True)

# remove unneeded columns
extracted.drop(['3rd Conf # / Shipping', 'Scheduled Room', 'Room Change Today', 'A/R Account', 'Undelivered Messages', 'Channel'], axis=1, inplace=True)
# extracted = extracted.loc[extracted['Arrival Date'].between(datetime(2023, 1, 1), datetime(2023, 6, 30), inclusive='both')]

# split and mend Arrival/Departure dates
InfoHMS_chunks = DataFrameWrapper(extracted).to_chunks(chunk_size=5_000)
InfoHMS_chunks: list[pd.DataFrame] = Parallel(n_jobs=-1, verbose=0)(delayed(mend_arrival_departure_dates)(chunk) for chunk in InfoHMS_chunks)
extracted = pd.concat(InfoHMS_chunks).sort_values('Arrival Date').reset_index(drop=True)

In [ ]:
df = extracted.copy(deep=True)

In [ ]:
InfoHMS_consolidated.loc[InfoHMS_consolidated['Arrival Date'].between(datetime(2023, 1, 1), datetime(2023, 6, 30), inclusive='both')]['Nights'].sum() - extracted['Split Nights'].sum()

-2948

In [175]:
df.loc[
    (df['Market Segment'] == 'Corp Group(Rooms w Meeting)') & 
    (df['Arrival Date'].dt.year == 2023) & 
    (df['Arrival Date'].dt.month == 2)
][['Arrival Date', 'Departure Date', 'Split Nights']]

,Arrival Date,Departure Date,Split Nights
2598,2023-02-01,2023-02-03,2
2601,2023-02-01,2023-02-03,2
2610,2023-02-01,2023-02-03,2
2618,2023-02-01,2023-02-03,2
2625,2023-02-01,2023-02-03,2
2629,2023-02-01,2023-02-03,2
2633,2023-02-01,2023-02-03,2
2734,2023-02-01,2023-02-03,2
2741,2023-02-01,2023-02-03,2
2745,2023-02-01,2023-02-03,2


In [ ]:
df = InfoHMS_consolidated.copy(deep=True)

tax_percentage = np.array(group_thresholds(df['Arrival Date'].dt.year, taxes))
df['Split Rate Grand Total'] = df['Split Rate Grand Total'] * (1 / tax_percentage)

bf_cost = group_categories(df['Non-Room Bundle'], breakfast_groups, filler = 0)

# set adults to 1, nights from 0 to 1 if bundle is '6BEERS'
bf_adults = np.array([1 if row['Non-Room Bundle'] == '6BEERS' else row['Adults'] for _, row in df.iterrows()])
bf_nights = df['Split Nights'].apply(lambda x: x if x > 0 else 1)

bf_cost = np.array(bf_cost) * bf_nights * bf_adults
df['Split Rate Grand Total'] = df['Split Rate Grand Total'] - bf_cost

In [ ]:
for mktseg in sorted(df['Market Segment'].unique()):
    revenue = df.loc[
        (df['Arrival Date'].between(datetime(2023, 2, 1), datetime(2023, 2, 28), inclusive='both')) & 
        (df['Market Segment'] == mktseg) & 
        (df['Status'] != 'No Show') & (df['Status'] != 'Cancelled')
    ]['Split Rate Grand Total'].sum()

    print(f'{mktseg: <30}{revenue}')

Airline Crew                  0
Airlines                      0
Complimentary                 0.0
Corp Group (Rooms Only)       26539.0
Corp Group(Rooms w Meeting)   594.0000000000001
Corporate FIT                 343703.75718637614
Corporate Long Stay           3493.627525704933
Corporate Volume              4261.0
House-use                     0.0
Internet                      920508.6599975992
Leisure FIT                   260058.5483832706
Lifetime Holidays (Member)    0.0
Shipping                      194794.88696960686
Wholesaler TA FIT             4636.560606060606
Wholesaler TA Group           6285.989898989899


In [167]:
df.loc[
    (df['Market Segment'] == 'Corp Group(Rooms w Meeting)') & 
    (df['Arrival Date'].dt.year == 2023) & 
    (df['Arrival Date'].dt.month == 2)
][['Arrival Date', 'Departure Date', 'Split Nights']]

,Arrival Date,Departure Date,Split Nights
158720,2023-02-01,2023-02-03,2
158722,2023-02-01,2023-02-03,2
158723,2023-02-01,2023-02-03,2
158812,2023-02-01,2023-02-03,2
158823,2023-02-01,2023-02-03,2
158824,2023-02-01,2023-02-03,2
158864,2023-02-01,2023-02-03,2
158868,2023-02-01,2023-02-03,2
158870,2023-02-01,2023-02-03,2
158871,2023-02-01,2023-02-03,2


In [52]:
breakfast_groups = {
    14 + 20 + 18: ['MUSETA'],
    12 + 14.94 + 18: ['MUSETB'],
    14: ['Internal ABF', 'Internal ABF - Bulk Buy'],
    9: ['Club BF'],
    8 + 14 + 12: ['SHN / PCA'],
    2 + 3.5 + 3: ['McDonald PCA'],
    13: ['High Tea'],
    17.5: ['Dayuse - 3 course Set Lunch'],
    39: ['6BEERS']
}

In [53]:
bf_cost = group_categories(df['Non-Room Bundle'], breakfast_groups, filler = 0)
bf_adults = np.array([row['Adults'] if row['Non-Room Bundle'] != '6BEERS' else 1 for _, row in df.iterrows()])
bf_nights = 
bf_cost = np.array(bf_cost) * df['Split Nights'] * bf_adults

df['BF Cost'] = bf_cost

In [58]:
df.loc[df['Non-Room Bundle'] == '6BEERS']['BF Cost'].unique()

array([ 0., 39.])

In [42]:
bf_cost = group_categories(df['Non-Room Bundle'], breakfast_groups, filler = 0)
bf_cost = np.array(bf_cost) * df['Split Nights'] * df['Adults']

In [37]:
df.loc[df['Split Rate Grand Total'] == bf_cost]['Nights'].unique()

array([  1,   2,   3,   7,   0,  11,   4,   5,   8,   6,  18,   9,  10,
        14,  12,  24,  13,  19,  41,  29,  42,  16,  20,  32,  30,  15,
        28, 120,  77,  17,  27,  25,  22,  23,  21,  26, 213,  64, 211,
        63, 208,  61,  52,  50,  76,  57,  55,  51,  74, 199,  35, 198,
        49,  33,  48,  39,  53,  46, 196,  45,  47,  44,  43,  40,  36,
        38,  37, 186,  60,  34,  31,  59, 142, 140,  56, 139, 138, 137,
        54, 136, 135, 132, 130, 126, 125,  89, 124, 122, 118, 116, 114,
       112, 110, 108, 106, 104, 101,  99,  98,  97,  96,  95,  94,  92,
        91,  90,  88,  87,  85,  83,  68,  81,  79,  78,  62,  75,  73,
        72,  71,  70,  69,  67,  66, 153,  93, 159], dtype=int64)

In [22]:
df['Rate Grand Total'] -= np.array(bf) * df['Adults'] * df['Nights']

In [26]:
df.loc[df['Rate Grand Total'] == 0]['Market Segment'].unique()

array(['Complimentary', 'Lifetime Holidays (Member)', 'Corporate FIT',
       'House-use', 'Leisure FIT', 'Wholesaler TA Group',
       'Wholesaler TA FIT', 'Corp Group(Rooms w Meeting)', 'Airlines',
       'Corporate Volume', 'Internet'], dtype=object)

In [17]:
np.array(bf) * df['Adults']

0         NaN
1         104
2          52
3         104
4         104
         ... 
177892    NaN
177893     42
177894     28
177895     14
177896     14
Name: Adults, Length: 177897, dtype: object

In [18]:
pd.isna(np.array(bf) * df['Adults']).sum()

84015